In [1]:
from numpy import info
import requests
from requests import Request, Session

import io
import pandas as pd
from datetime import datetime
import json
from aioconnect.helpers import *
from dotenv import load_dotenv
from os import getenv

In [2]:
import aioconnect

In [3]:
def get_token(
    email: str,
    password: str,
):
    """Log into AIO Impact and get a token.

    Parameters
    ----------
    email : str
        Email address of the user account.

    password : str
        Password of the user account.

    Returns
    -------
    str
        Bearer token.

    Raises
    -------
    requests.exceptions.HTTPError
        If the username and password are wrong.

    Examples
    --------
    >>> aioconnect.get_token(
    >>>     email="firstname.lastname@aioneers.com", password="xxx",
    >>> )
    """

    url = getenv("CONNECT_URL") + "/login"
    payload = json.dumps({"email": email, "password": password})
    headers = {"Content-Type": "application/json"}

    response = requests.request("POST", url, headers=headers, data=payload)
    response.raise_for_status()

    token = response.json()["token"]

    return token

In [4]:
def test_correct_credentials():
    password = aioconnect.vault_get_secret(
        scope="aio-data-science-key", key="sebastian-szilvas-aio-impact"
    )

    res = get_token(
        email="sebastian.szilvas@aioneers.com",
        password=f"{password}",
    )

    assert isinstance(res, str)
    assert len(res) > 250

def test_wrong_password():
    try:
        res = get_token(
            email="sebastian.szilvas@aioneers.com",
            password="wrong password",
        )
    except requests.exceptions.HTTPError as exception:
        assert exception.response.status_code == 401


In [5]:
test_correct_credentials()
test_wrong_password()

In [18]:
def upsert_DOT(
    token: str,
    dataframe: pd.DataFrame,
) -> list:
    """Create a new DOT in AIO Impact or update it if the DOT is already existing.

    Parameters
    ----------
    token : str
        Token which was returned from the user login.

    dataframe : Pandas.DataFrame
        Dataframe containing DOT details.

    Returns
    -------
    requests.Response
        HTTP response.

    Examples
    --------
    >>> token = aioconnect.get_token(
    >>> email="firstname.lastname@aioneers.com", password="xxx",
    >>> )
    >>> res = aioconnect.upsert_DOT(
    >>>     token=token,
    >>>     dataframe = df
    >>> )
    """

    url = getenv("CONNECT_URL") + "/dots"

    columns_list = [
        "externalId",
        "name",
        "description",
        "metricId",
        "actuals",
    ]

    if all([item in dataframe.columns for item in columns_list]):
        payload = dataframe.to_json(orient="records")

        response = requests.put(
        url=url, headers={"Authorization": f"Bearer {token}",
                            'Content-Type': 'application/json'}, data=payload)
        response.raise_for_status()

        return response
    else:
        raise KeyError("Columns not correct")

In [23]:
password = aioconnect.vault_get_secret(
        scope="aio-data-science-key", key="sebastian-szilvas-aio-impact"
    )

token = get_token(
    email="sebastian.szilvas@aioneers.com",
    password=f"{password}",
)


def test_single_DOT_correct_headers():
    data = {
        "externalId": ["2343284"],
        "name": ["helloooooo"],
        "description": "sdklfjslfdjsldf",
        "metricId": 10,
        "actuals": [[
            {
                    "timestamp": "2021-12-07T14:59:38.273Z",
                    "value": 788237
            }
        ]],
    }
    
    df = pd.DataFrame.from_dict(data)

    res = upsert_DOT(
        token=token, dataframe=df)

    assert res.status_code == 200
    assert (res.json()["created"] == len(df.index) or
            res.json()["updated"] == len(df.index))


def test_two_DOTs_correct_headers():
    data = {
                "externalId": ["2343284","73894792"],
                "name": ["helloooooo","helloooooo2"],
                "description": ["sdklfjslfdjsldf","sdklfjslfdjsldf"],
                "metricId": [10,10],
                "actuals": [[
                    {
                            "timestamp": "2021-12-07T14:59:38.273Z",
                            "value": 15
                    }
                ],
                [
                    {
                            "timestamp": "2021-12-07T14:59:38.273Z",
                            "value": 17
                    }
                ]
                ],
            }
    df = pd.DataFrame.from_dict(data)

    res = upsert_DOT(token = token,
        dataframe= df,
    )

    assert res.status_code == 200
    assert (res.json()["created"] == len(df.index) or
            res.json()["updated"] == len(df.index))

In [24]:
test_single_DOT_correct_headers()
test_two_DOTs_correct_headers()

  externalId         name      description  metricId  \
0    2343284   helloooooo  sdklfjslfdjsldf        10   
1   73894792  helloooooo2  sdklfjslfdjsldf        10   

                                             actuals  
0  [{'timestamp': '2021-12-07T14:59:38.273Z', 'va...  
1  [{'timestamp': '2021-12-07T14:59:38.273Z', 'va...  


# This function still needs fixing

In [ ]:
# def get_object(
#     token: str,
#     object: str = "digitalObjectTwins",
# ) -> list:
#     """Get JSON object.

#     Parameters
#     ----------
#     token : str
#         Token which was returned from the user login.
#     object : str = "dotTypes"
#         Object to be extracted from the API.

#     Returns
#     -------
#     list
#         List of JSON objects.

#     Raises
#     ------
#     KeyError
#         Raises KeyError when the input is not correct.
#     """
#     url = getenv("CONNECT_URL")

#     if object == "digitalObjectTwins":
#         url += "/digitalObjectTwins/"
#     elif object == "measures":
#         url += "/measures/"
#     elif object == "initiatives":
#         url += "/initiatives/"
#     else:
#         raise KeyError

#     headers = {"Authorization": f"Bearer {token}"}

#     response = requests.request("GET", url, headers=headers)
#     response.raise_for_status()
#     return response.json()